In [1]:
import json, time
from IPython.display import clear_output
from STservo import ST3215, Calibrator 

class MAPPER:
    def __init__(self):
        with open('Controller.json', 'r') as fr: 
            conf_ctrl = json.load(fp=fr)
        with open('Arm.json'       , 'r') as fr:
            conf_arm = json.load(fp=fr)
        
        self.dev_id = list(range(1, 13))
        for id_int in self.dev_id:
            #continue
            id_str = str(id_int)
            if conf_ctrl[id_str]['min'] > 2047:
                conf_ctrl[id_str]['min'] = conf_ctrl[id_str]['min'] - conf_ctrl[id_str]['corr'] - 4096
            else:
                conf_ctrl[id_str]['min'] = conf_ctrl[id_str]['min'] - conf_ctrl[id_str]['corr']
            if conf_arm [id_str]['min'] > 2047:
                conf_arm[ id_str]['min'] = conf_arm [id_str]['min'] - conf_arm [id_str]['corr'] - 4096
            else:
                conf_arm[ id_str]['min'] = conf_arm [id_str]['min'] - conf_arm [id_str]['corr']

        self.conf_ctrl = conf_ctrl
        self.conf_arm  = conf_arm 

    def getPosiVelo(self, result):
        arm_dict = {}
        for id_int in self.dev_id:
            id_str = str(id_int)
            ratio  = (result[id_int]['posi'] - self.conf_ctrl[id_str]['min']) / self.conf_ctrl[id_str]['range']
            if id_int == 12:
                ratio = 1-ratio
            posi   = self.conf_arm[id_str]['min'] + self.conf_ctrl[id_str]['range'] * ratio
                
            arm_dict[id_str] = {'posi': int(posi), 'velo': 1200}
        arm_json = json.dumps(arm_dict)
        return len(arm_json).to_bytes(length=2, byteorder='big') + arm_json.encode()

m = MAPPER()
m.conf_arm, m.conf_ctrl

({'1': {'min': 100, 'range': 2721, 'corr': -1479},
  '2': {'min': 100, 'range': 2403, 'corr': -692},
  '3': {'min': 100, 'range': 2209, 'corr': -252},
  '4': {'min': 100, 'range': 2350, 'corr': -1732},
  '5': {'min': 100, 'range': 3815, 'corr': -1736},
  '6': {'min': 100, 'range': 1489, 'corr': -1104},
  '7': {'min': 100, 'range': 2728, 'corr': -2036},
  '8': {'min': 100, 'range': 2417, 'corr': -1329},
  '9': {'min': 100, 'range': 2215, 'corr': 1748},
  '10': {'min': 100, 'range': 2389, 'corr': 1426},
  '11': {'min': 100, 'range': 3817, 'corr': 64},
  '12': {'min': 100, 'range': 1493, 'corr': -1608}},
 {'1': {'min': 100, 'range': 2718, 'corr': -1666},
  '2': {'min': 100, 'range': 2400, 'corr': 1345},
  '3': {'min': 100, 'range': 2225, 'corr': 351},
  '4': {'min': 100, 'range': 2315, 'corr': 488},
  '5': {'min': 100, 'range': 3803, 'corr': 1428},
  '6': {'min': 37, 'range': 1268, 'corr': -2047},
  '7': {'min': 100, 'range': 2710, 'corr': 883},
  '8': {'min': 100, 'range': 2397, 'corr': 

In [ ]:
import socket

servo_ctrl = ST3215(port='/dev/ttyACM0')

HOST = '192.168.3.110'   # 服务器地址
PORT = 5000              # 服务器端口
try:
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((HOST, PORT))
    while True:
        #time.sleep(1)
        result = servo_ctrl.readPosi(id_arr=m.dev_id, debug=True)
        client.sendall(m.getPosiVelo(result))
        assert b'\xff\xff' == client.recv(2)
            
except KeyboardInterrupt:
    client.sendall(b'\x00\x00')
    pass 

In [ ]:
result